# Pinecone Serverless Reranking – Projet complet en français

**Objectif :** Améliorer la pertinence des résultats de recherche grâce au reranking par modèle. Ce projet utilise Pinecone, FAISS, Sentence Transformers et une API serverless.

# Installation des bibliothèques

In [ ]:
pip install pinecone==6.0.1 pinecone-notebooks pandas torch transformers sentence-transformers faiss-cpu

: 

# Authentification avec Pinecone

In [ ]:
import os
if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

# Initialiser le client Pinecone

In [ ]:
from pinecone import Pinecone
api_key = os.environ["PINECONE_API_KEY"]
environment = "us-west1-gcp"  # Remplace par ton environnement
pc = Pinecone(api_key=api_key, environment=environment)

# Reranking simple : pomme (fruit) vs Apple Inc.

In [ ]:
query = "Tell me about Apple's products"
documents = [
    "Apple is a fruit rich in fiber and vitamins.",
    "Apple Inc. is famous for its iPhone and Mac computers.",
    "I planted an apple tree in my garden.",
    "Apple’s latest keynote introduced new AI features.",
    "The apple pie was delicious at grandma’s house."
]

In [ ]:
from pinecone import RerankModel
reranked = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
    top_n=3
)

In [ ]:
def show_reranked(query, matches):
    print(f"Query: {query}")
    for i, m in enumerate(matches):
        print(f"{i+1}. Score: {round(m.score, 3)} | {m.document.text}")
show_reranked(query, reranked.matches)

# Création d’un index serverless Pinecone

In [ ]:
from pinecone import ServerlessSpec
cloud = "aws"
region = "us-east-1"
spec = ServerlessSpec(cpu=1, memory_gb=1)
index_name = "pinecone-reranker"
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], environment=f"{cloud}-{region}")

In [ ]:
if pc.has_index(index_name):
    pc.delete_index(index_name)
pc.create_index(name=index_name, dimension=384, serverless_config=spec)

# Charger des notes médicales avec embeddings

In [ ]:
import requests, tempfile, pandas as pd
with tempfile.TemporaryDirectory() as tmpdir:
    file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")
    url = "https://raw.githubusercontent.com/pinecone-io/examples/main/notebooks/semantic-search-reranking/data/sample_notes_data.jsonl"
    resp = requests.get(url)
    open(file_path, "wb").write(resp.content)
    df = pd.read_json(file_path, orient='records', lines=True)
df.head()

# Insérer les vecteurs dans Pinecone

In [ ]:
index = pc.Index(index_name)
index.upsert_from_dataframe(df)

import time
def is_ready(idx):
    stats = idx.describe_index_stats()
    return stats.total_vector_count > 0

while not is_ready(index):
    time.sleep(5)
print(index.describe_index_stats())

# Recherche sémantique avec embedding

In [ ]:
from sentence_transformers import SentenceTransformer
def get_embedding(text):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    return model.encode(text)

question = "what if my patient has leg pain"
emb = get_embedding(question)
results = index.query(vector=emb, top_k=5, include_metadata=True)
matches = sorted(results.matches, key=lambda m: m.score, reverse=True)

In [ ]:
def show_results(q, matches):
    print(f"Question: {q}")
    for i, m in enumerate(matches):
        print(f"{i+1}. ID: {m.id} | Score: {round(m.score, 2)} | Meta: {m.metadata}")
show_results(question, matches)

# Reranking contextuel des notes médicales

In [ ]:
rerank_docs = [
    {"id": m.id, "reranking_field": "; ".join([f"{k}: {v}" for k, v in m.metadata.items()])}
    for m in matches
]
rerank_query = "patient pain in leg after surgery"
reranked = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=rerank_query,
    documents=rerank_docs,
    rank_fields=["reranking_field"],
    top_n=3
)

def show_reranked(q, matches):
    print(f"Refined Query: {q}")
    for i, m in enumerate(matches):
        print(f"{i+1}. ID: {m.document.id} | Score: {round(m.score, 2)} | {m.document.reranking_field}")
show_reranked(rerank_query, reranked.matches)


Ce projet démontre comment l'intégration d'un système de reranking par similarité sémantique améliore significativement la pertinence des résultats de recherche dans un contexte métier. Grâce à Pinecone, nous avons pu :

-Indexer efficacement un corpus (comme des notes médicales) à l’aide d’un vecteur store serverless,
-Appliquer une première recherche sémantique via des embeddings pour trouver les documents les plus proches,
-Utiliser un modèle de reranking avancé (bge-reranker-v2-m3) pour réordonner les résultats en fonction d’une requête plus précise et du contexte métier (comme les symptômes ou antécédents d’un patient).